# Evaluation of the first model and save pseudo labels
This notebook is for getting pseudo labels and save them to train the second model. Since the first model is trained with grayscale images, we use grayscale images here too to get the pseudo labels.

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from options.test_options import TestOptions
from datagray import CreateTrgDataLoader
from datagray import CreateTestDataLoader
from datagray import CreateTrgDataLoader_trainset
from PIL import Image
import json
import os.path as osp
import os
import numpy as np
from model import CreateModel
from utils import FDA_source_to_target

In [2]:
palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)
for i in range(zero_pad):
    palette.append(0)

## Test options
Adjust the directories to load ground truth labels and save the results. Also, parameter beta can be adjusted here which will be used for translation of target images to source domain.

In [3]:
import argparse
import os.path as osp
class TestOptions():
    def initialize(self):
        parser = argparse.ArgumentParser(description="test segmentation network")
        parser.add_argument("--model", type=str, default='DeepLab', help="available options : DeepLab,VGG and ENet")
        parser.add_argument("--GPU", type=str, default='0', help="which GPU to use")
        parser.add_argument("--source", type=str, default='gta5', help="source dataset : gta5 or synthia")
        parser.add_argument("--data-dir-target", type=str, default='../data/cityscapes', help="Path to the directory containing the target dataset.")
        parser.add_argument("--data-list-target", type=str, default='./dataset/cityscapes_list/train.txt', help="list of images in the target dataset.")
        parser.add_argument("--data-dir-test", type=str, default='../data/GTA5', help="Path to the directory containing the source dataset.")
        parser.add_argument("--data-list-test", type=str, default='./dataset/gta5_list/train_all.txt', help="Path to the listing of images in the source dataset.")
        parser.add_argument("--LB", type=float, default=0.01, help="beta for FDA")
        parser.add_argument("--num-classes", type=int, default=19, help="Number of classes for cityscapes.")
        parser.add_argument("--set", type=str, default='val', help="choose test set.")
        # parser.add_argument("--set", type=str, default='train', help="choose test set.")
        parser.add_argument("--restore-opt1", type=str, default=None, help="restore model parameters from beta1")
        parser.add_argument("--restore-opt2", type=str, default=None, help="restore model parameters from beta2")
        parser.add_argument("--restore-opt3", type=str, default=None, help="restore model parameters from beta3")

        parser.add_argument("--init-weights", type=str, default=None, help="initial model.")
        parser.add_argument("--restore-from", type=str, default='../checkpoints/UDA_ENet_val/new_arch_cont/gta5_100000', help="restore model parameters from")

        parser.add_argument("--save", type=str, default='../results/pseudo_labels', help="Path to save result.")
        parser.add_argument('--gt_dir', type=str, default='../data/cityscapes/gtFine/train', help='directory for CityScapes train gt images')
        parser.add_argument('--devkit_dir', type=str, default='./dataset/cityscapes_list', help='list directory of cityscapes')         

        return parser.parse_args(args=[])

In [4]:
opt = TestOptions()
args = opt.initialize()


os.environ["CUDA_VISIBLE_DEVICES"] = args.GPU

os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
memory_gpu=[int(x.split()[2]) for x in open('tmp','r').readlines()]
os.system('rm tmp')    
os.environ["CUDA_VISIBLE_DEVICES"] = str(np.argmax(memory_gpu))  

if not os.path.exists(args.save):
    os.makedirs(args.save)
 
       
model = CreateModel(args)   
model.eval()
model.cuda() 
 
targetloader = CreateTrgDataLoader_trainset(args) # Use CreateTrgDataLoader_trainset to save pseudo labels

sourceloader = CreateTestDataLoader(args)
sourceloader_iter = iter(sourceloader)

Definitions of the functions to be used for calculation of mIoU and visualization of predictions. 

In [21]:
def colorize_mask(mask):
    # mask: numpy array of the mask
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)

    return new_mask
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape(n, n)


def per_class_iu(hist):
    return np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))


def label_mapping(input, mapping):
    output = np.copy(input)
    for ind in range(len(mapping)):
        output[input == mapping[ind][0]] = mapping[ind][1]
    return np.array(output, dtype=np.int64)

def compute_mIoU(gt_dir, pred_dir, devkit_dir='', restore_from=''):
    with open(osp.join(devkit_dir, 'info.json'), 'r') as fp:
        info = json.load(fp)
    num_classes = np.int(info['classes'])
    print('Num classes', num_classes)
    name_classes = np.array(info['label'], dtype=np.str)
    mapping = np.array(info['label2train'], dtype=np.int)
    hist = np.zeros((num_classes, num_classes))

    image_path_list = osp.join(devkit_dir, 'train.txt')
    label_path_list = osp.join(devkit_dir, 'train.txt')
    
    gt_imgs = open(label_path_list, 'r').read().splitlines()
    gt_imgs = [osp.join(gt_dir, x) for x in gt_imgs]
    pred_imgs = open(image_path_list, 'r').read().splitlines()
    pred_imgs = [osp.join(pred_dir, x.split('/')[-1]) for x in pred_imgs]
    
    for ind in range(len(gt_imgs)):
        pred = np.array(Image.open(pred_imgs[ind]))
        
        lbname = gt_imgs[ind].replace("leftImg8bit", "gtFine_labelIds")
        
        label = np.array(Image.open(lbname))
        
        label = label_mapping(label, mapping)
        if len(label.flatten()) != len(pred.flatten()):
            print('Skipping: len(gt) = {:d}, len(pred) = {:d}, {:s}, {:s}'.format(len(label.flatten()), len(pred.flatten()), gt_imgs[ind], pred_imgs[ind]))
            continue
        hist += fast_hist(label.flatten(), pred.flatten(), num_classes)
        if ind > 0 and ind % 10 == 0:
            with open(restore_from+'_mIoU.txt', 'a') as f:
                f.write('{:d} / {:d}: {:0.2f}\n'.format(ind, len(gt_imgs), 100*np.mean(per_class_iu(hist))))
            print('{:d} / {:d}: {:0.2f}'.format(ind, len(gt_imgs), 100*np.mean(per_class_iu(hist))))
    hist2 = np.zeros((19, 19))
    for i in range(19):
        hist2[i] = hist[i] / np.sum(hist[i])
    
    mIoUs = per_class_iu(hist)
    for ind_class in range(num_classes):
        with open(restore_from+'_mIoU.txt', 'a') as f:
            f.write('===>' + name_classes[ind_class] + ':\t' + str(round(mIoUs[ind_class] * 100, 2)) + '\n')
        print('===>' + name_classes[ind_class] + ':\t' + str(round(mIoUs[ind_class] * 100, 2)))
    with open(restore_from+'_mIoU.txt', 'a') as f:
        f.write('===> mIoU: ' + str(round(np.nanmean(mIoUs) * 100, 2)) + '\n')
    print('===> mIoU19: ' + str(round(np.nanmean(mIoUs) * 100, 2)))
    print('===> mIoU16: ' + str(round(np.mean(mIoUs[[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 17, 18]]) * 100, 2)))
    print('===> mIoU13: ' + str(round(np.mean(mIoUs[[0, 1, 2, 6, 7, 8, 10, 11, 12, 13, 15, 17, 18]]) * 100, 2)))   

Running this cell will get the predictions from the given model, save them as pseudo labels and calculate mIoU. Since the model trained with original source images, we will transfer target images to source domain.

In [6]:
for index, batch in enumerate(targetloader):
    if index % 100 == 0:
        print ('%d processd' % index)
    image, _, name = batch
    src_img, src_lbl, _, _ = sourceloader_iter.next()
    image = FDA_source_to_target(image, src_img, L=args.LB)
    output = model(Variable(image).cuda())
    output = nn.functional.softmax(output, dim=1)
    output = nn.functional.upsample(output, (1024, 2048), mode='bilinear', align_corners=True).cpu().data[0].numpy()
    output = output.transpose(1,2,0)
    output_nomask = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)
    output_col = colorize_mask(output_nomask)
    output_nomask = Image.fromarray(output_nomask)    
    name = name[0].split('/')[-1]
    output_nomask.save('%s/%s' % (args.save, name))
    output_col.save('%s/%s_color.png' % (args.save, name.split('.')[0])) 
        
compute_mIoU(args.gt_dir, args.save, args.devkit_dir, args.restore_from)    


0 processd


/home/singh/anaconda3/envs/adl4cv/lib/python3.8/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


100 processd
200 processd
300 processd
400 processd
500 processd
600 processd
700 processd
800 processd
900 processd
1000 processd
1100 processd
1200 processd
1300 processd
1400 processd
1500 processd
1600 processd
1700 processd
1800 processd
1900 processd
2000 processd
2100 processd
2200 processd
2300 processd
2400 processd
2500 processd
2600 processd
2700 processd
2800 processd
2900 processd
Num classes 19


<ipython-input-5-ca54800ec30d>:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  num_classes = np.int(info['classes'])
<ipython-input-5-ca54800ec30d>:27: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(info['l

FileNotFoundError: [Errno 2] No such file or directory: '../data/cityscapes/gtFine/train/frankfurt/frankfurt_000001_007973_gtFine_labelIds.png'

In [22]:
compute_mIoU(args.gt_dir, args.save, args.devkit_dir, args.restore_from)    

<ipython-input-21-5440c131f434>:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  num_classes = np.int(info['classes'])
<ipython-input-21-5440c131f434>:27: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  name_classes = np.array(info[

Num classes 19


<ipython-input-21-5440c131f434>:13: RuntimeWarning: invalid value encountered in true_divide
  return np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))


10 / 2975: nan
20 / 2975: 42.78
30 / 2975: 41.22
40 / 2975: 41.82
50 / 2975: 42.21
60 / 2975: 42.34
70 / 2975: 42.22
80 / 2975: 42.21
90 / 2975: 42.20
100 / 2975: 41.65
110 / 2975: 41.59
120 / 2975: 41.67
130 / 2975: 41.38
140 / 2975: 41.64
150 / 2975: 42.03
160 / 2975: 41.43
170 / 2975: 41.12
180 / 2975: 44.36
190 / 2975: 44.62
200 / 2975: 45.08
210 / 2975: 45.22
220 / 2975: 45.05
230 / 2975: 45.00
240 / 2975: 45.11
250 / 2975: 45.15
260 / 2975: 45.66
270 / 2975: 45.49
280 / 2975: 44.46
290 / 2975: 44.40
300 / 2975: 44.53
310 / 2975: 44.61
320 / 2975: 44.77
330 / 2975: 44.96
340 / 2975: 44.94
350 / 2975: 44.50
360 / 2975: 44.86
370 / 2975: 45.15
380 / 2975: 45.37
390 / 2975: 45.27
400 / 2975: 45.30
410 / 2975: 45.35
420 / 2975: 45.49
430 / 2975: 45.60
440 / 2975: 45.61
450 / 2975: 45.60
460 / 2975: 45.53
470 / 2975: 45.48
480 / 2975: 45.44
490 / 2975: 45.51
500 / 2975: 45.54
510 / 2975: 45.55
520 / 2975: 45.60
530 / 2975: 45.70
540 / 2975: 45.69
550 / 2975: 45.71
560 / 2975: 45.69
570